# g) Ngonye Falls Power and Energy Periods
Produces 'simulations' of station operation over two periods, debt period and equity period. Each simulation has a different start date on the historic series.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
|    [Model Name]_pe_yearly.csv| Notebook: d_ewr_setup | Summary energy produciton by year of the historic series |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
| [Model Name]_pe_sims.csv | Detailed (yearly) output. |
| [Model Name]_pe_sims_summary.csv | Summary by start year  |



## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [2]:
input_data='./input_data/'
output_data='./output_data/'

model_name='Base Case'


In [3]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix

debt_period=18
equity_period=25
discount_rate=1.0 + 0.15

In [4]:
yearly=pd.read_csv(output_data + out_prefix + "_pe_yearly.csv")

In [5]:
yearly

,WaterYear,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor,Prop4Units,Prop3Units,Prop2Units,Prop1Units,Prop0Units,Days0Units,Exceedance
0,1924,3452.248382,111.407480,0.870678,0.053009,25.407672,13.492193,974.570905,964.398502,190.504540,0.000000,827895.372600,0.525048,0.408219,0.035616,0.109589,0.419178,0.027397,10,0.60
1,1925,4501.386001,158.541897,0.484050,0.040654,25.354758,11.555578,976.507520,964.398502,192.161479,22.641758,770213.258953,0.488466,0.317808,0.076712,0.189041,0.416438,0.000000,0,0.85
2,1926,3313.040641,192.895237,0.497958,0.065499,25.437890,13.777038,974.286060,964.398502,191.126353,20.615206,859283.496984,0.544954,0.380822,0.057534,0.178082,0.383562,0.000000,0,0.35
3,1927,2261.108088,179.826510,0.472707,0.065455,25.390319,16.221674,971.841425,964.398502,190.628451,22.732948,827169.597253,0.524588,0.304110,0.106849,0.178082,0.413699,0.000000,0,0.60
4,1928,1589.152359,195.207704,0.498797,0.093131,25.387451,18.159351,969.903747,964.407470,193.051045,23.518345,747422.292641,0.474012,0.172603,0.147945,0.208219,0.471233,0.000000,0,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2012,3614.802471,223.273423,0.508220,0.055826,25.083495,13.168827,974.894271,964.716509,191.310490,24.528540,843130.264390,0.534710,0.410959,0.079452,0.126027,0.383562,0.000000,0,0.45
89,2013,3574.946119,195.061590,0.497279,0.056428,25.436464,13.247224,974.815874,964.405522,191.927167,20.671377,926459.111352,0.587557,0.446575,0.131507,0.128767,0.293151,0.000000,0,0.10
90,2014,1460.253290,208.780681,0.546606,0.099878,25.098362,18.623511,969.484512,964.706890,192.407900,25.316443,884558.950509,0.560984,0.309589,0.115068,0.161644,0.413699,0.000000,0,0.25
91,2015,2975.524395,176.218173,0.496598,0.067347,25.390405,14.500960,973.562138,964.405522,191.268651,21.964965,786286.526245,0.498660,0.323288,0.076712,0.164384,0.438356,0.000000,0,0.80


In [6]:
sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims['SimName']=sims['StartYear']
sims['EquivalentPeriod']=2027-sims['StartYear']-2
sims=sims.set_index('StartYear')
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod
StartYear,,,,
1924,1941,1948,1924,101
1925,1942,1949,1925,100
1926,1943,1950,1926,99
1927,1944,1951,1927,98
1928,1945,1952,1928,97
...,...,...,...,...
1988,2005,2012,1988,37
1989,2006,2013,1989,36
1990,2007,2014,1990,35


In [7]:
sims.loc[1967]

DebtEnd             1984
EquityEnd           1991
SimName             1967
EquivalentPeriod      58
Name: 1967, dtype: int64

In [8]:


o=pd.DataFrame()
for row in sims.itertuples(): 
    f=pd.DataFrame(yearly.loc[(yearly['WaterYear']>=row.Index) & (yearly['WaterYear']<=row.EquityEnd)].loc[:,['Energy']])
    f['Energy']=f['Energy']/1000
    f=f.reset_index()
    f['YearNo']=f.index+1
    f['StartYear']=row.Index
    f['WaterYear']=f['StartYear']+f.index
    f=f.set_index('StartYear').drop('index',axis=1)
    #display(f)
    
    o=o.append(f)

sims=sims.join(o)
#sims=sims.reset_index()
#sims=sims.set_index(['StartYear','YearNo'])
sims
#o

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear
StartYear,,,,,,,
1924,1941,1948,1924,101,827.895373,1,1924
1924,1941,1948,1924,101,770.213259,2,1925
1924,1941,1948,1924,101,859.283497,3,1926
1924,1941,1948,1924,101,827.169597,4,1927
1924,1941,1948,1924,101,747.422293,5,1928
...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,843.130264,21,2012
1992,2009,2016,1992,33,926.459111,22,2013
1992,2009,2016,1992,33,884.558951,23,2014


In [9]:
flow_yearly=pd.read_csv(output_data + "ngonye_flow_yearly.csv").set_index('WaterYear')
sims['Flow_mean']=sims.join(flow_yearly,on='WaterYear',lsuffix='l_')['Flow_mean']
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean
StartYear,,,,,,,,
1924,1941,1948,1924,101,827.895373,1,1924,992.089057
1924,1941,1948,1924,101,770.213259,2,1925,1111.873684
1924,1941,1948,1924,101,859.283497,3,1926,990.425137
1924,1941,1948,1924,101,827.169597,4,1927,848.055028
1924,1941,1948,1924,101,747.422293,5,1928,564.473244
...,...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,843.130264,21,2012,1336.562700
1992,2009,2016,1992,33,926.459111,22,2013,1340.578429
1992,2009,2016,1992,33,884.558951,23,2014,697.100110


In [10]:
sims['CumEnergy']=sims['Energy'].groupby('StartYear').cumsum()

In [11]:
sims['DiscountedEnergy']=sims['Energy']*(1/(discount_rate**(sims['YearNo']-1)))

In [12]:
sims['CumDiscountedEnergy']=sims['DiscountedEnergy'].groupby('StartYear').cumsum()

In [13]:
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean,CumEnergy,DiscountedEnergy,CumDiscountedEnergy
StartYear,,,,,,,,,,,
1924,1941,1948,1924,101,827.895373,1,1924,992.089057,827.895373,827.895373,827.895373
1924,1941,1948,1924,101,770.213259,2,1925,1111.873684,1598.108632,669.750660,1497.646033
1924,1941,1948,1924,101,859.283497,3,1926,990.425137,2457.392129,649.741775,2147.387807
1924,1941,1948,1924,101,827.169597,4,1927,848.055028,3284.561726,543.877437,2691.265244
1924,1941,1948,1924,101,747.422293,5,1928,564.473244,4031.984018,427.341122,3118.606366
...,...,...,...,...,...,...,...,...,...,...,...
1992,2009,2016,1992,33,843.130264,21,2012,1336.562700,16674.997573,51.515494,5342.998950
1992,2009,2016,1992,33,926.459111,22,2013,1340.578429,17601.456685,49.223400,5392.222350
1992,2009,2016,1992,33,884.558951,23,2014,697.100110,18486.015635,40.867145,5433.089495


In [14]:
sims_summary=(sims.loc[sims['YearNo']==debt_period]).drop(['Energy','YearNo','DiscountedEnergy','DebtEnd','EquityEnd'],axis=1).rename(columns={'CumEnergy':'DebtEnergy','CumDiscountedEnergy':'DebtDiscountedEnergy'})

sims_summary=sims_summary.join((sims.loc[sims['YearNo']==equity_period])
            .drop(['Energy','YearNo','DiscountedEnergy','SimName','EquivalentPeriod','WaterYear','Flow_mean'],axis=1)
            .rename(columns={'CumEnergy':'EquityEnergy','CumDiscountedEnergy':'EquityDiscountedEnergy'}))

sims_summary.head(3)

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy
StartYear,,,,,,,,,,
1924,1924,101,1941,649.518919,14854.005152,5782.855199,1941,1948,20404.913573,6093.784390
1925,1925,100,1942,669.368439,14835.094775,5773.379448,1942,1949,20334.518071,6082.235085
1926,1926,99,1943,927.855019,14910.324616,5832.204667,1943,1950,20410.124804,6138.373214


In [15]:
sims_summary['DebtAvEnergy']=sims.loc[sims['YearNo']<=debt_period]['Energy'].groupby('StartYear').mean()

In [16]:
sims_summary['EquityAvEnergy']=sims.loc[sims['YearNo']<=equity_period]['Energy'].groupby('StartYear').mean()

In [17]:
sims_summary['DebtAvFlow']=sims.loc[sims['YearNo']<=debt_period]['Flow_mean'].groupby('StartYear').mean()
sims_summary['EquityAvFlow']=sims.loc[sims['YearNo']<=equity_period]['Flow_mean'].groupby('StartYear').mean()

In [18]:
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow
StartYear,,,,,,,,,,,,,,
1924,1924,101,1941,649.518919,14854.005152,5782.855199,1941,1948,20404.913573,6093.784390,825.222508,816.196543,924.969173,932.862973
1925,1925,100,1942,669.368439,14835.094775,5773.379448,1942,1949,20334.518071,6082.235085,824.171932,813.380723,907.040250,948.130812
1926,1926,99,1943,927.855019,14910.324616,5832.204667,1943,1950,20410.124804,6138.373214,828.351368,816.404992,896.816991,952.625050
1927,1927,98,1944,996.690083,14847.016515,5792.826065,1944,1951,20357.323850,6099.127064,824.834251,814.292954,897.165043,976.692444
1928,1928,97,1945,680.775211,14891.584842,5791.511958,1945,1952,20345.400221,6091.231120,827.310269,813.816009,887.871720,1002.207598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1988,37,2005,960.131883,13697.227689,5259.499937,2005,2012,19735.282706,5592.202494,760.957094,789.411308,826.412437,1002.032194
1989,1989,36,2006,1457.112425,13691.234047,5157.392239,2006,2013,19819.345137,5494.641870,760.624114,792.773805,833.605851,1002.550092
1990,1990,35,2007,1300.199178,13835.408340,5152.686144,2007,2014,19954.938198,5488.428809,768.633797,798.197528,876.126697,1009.041138


In [19]:
fdc=pd.read_csv(output_data + "ngonye_flow_annual_exceedance.csv")
sims_summary['DebtEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('DebtAvFlow'),fdc.sort_values('Mean'),left_on='DebtAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary['EquityEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('EquityAvFlow'),fdc.sort_values('Mean'),left_on='EquityAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow,DebtEqExceed,EquityEqExceed
StartYear,,,,,,,,,,,,,,,,
1924,1924,101,1941,649.518919,14854.005152,5782.855199,1941,1948,20404.913573,6093.784390,825.222508,816.196543,924.969173,932.862973,0.653,0.649
1925,1925,100,1942,669.368439,14835.094775,5773.379448,1942,1949,20334.518071,6082.235085,824.171932,813.380723,907.040250,948.130812,0.657,0.636
1926,1926,99,1943,927.855019,14910.324616,5832.204667,1943,1950,20410.124804,6138.373214,828.351368,816.404992,896.816991,952.625050,0.660,0.630
1927,1927,98,1944,996.690083,14847.016515,5792.826065,1944,1951,20357.323850,6099.127064,824.834251,814.292954,897.165043,976.692444,0.660,0.587
1928,1928,97,1945,680.775211,14891.584842,5791.511958,1945,1952,20345.400221,6091.231120,827.310269,813.816009,887.871720,1002.207598,0.662,0.551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1988,37,2005,960.131883,13697.227689,5259.499937,2005,2012,19735.282706,5592.202494,760.957094,789.411308,826.412437,1002.032194,0.708,0.551
1989,1989,36,2006,1457.112425,13691.234047,5157.392239,2006,2013,19819.345137,5494.641870,760.624114,792.773805,833.605851,1002.550092,0.704,0.551
1990,1990,35,2007,1300.199178,13835.408340,5152.686144,2007,2014,19954.938198,5488.428809,768.633797,798.197528,876.126697,1009.041138,0.669,0.547


In [20]:
sims_summary.to_csv(output_data + out_prefix + "_pe_sims_summary.csv")

In [21]:
sims.to_csv(output_data + out_prefix + "_pe_sims.csv")